In [1]:
# 데이터 집계와 그룹 연산
## GroupBy 메카닉
# 그룹 연산 : 분리-적용-결합
#          분리 : 여러 객체에 들어 있는 데이터를 하나 이상의 key를 기준으로 분리한다.
#          적용 : 분리 후 함수를 각 그룹에 적용시켜 새로운 값을 얻어낸다.
#          결합 : 함수를 적용한 결과를 하나의 객체로 결합한다.

import pandas as pd
import numpy as np
from pandas import Series, DataFrame

df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                  'key2' : ['one', 'two', 'one', 'two', 'one'],
                  'data1' : np.random.randn(5),
                  'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-2.290685,0.133323
1,a,two,-0.085906,0.497789
2,b,one,0.513622,-1.075834
3,b,two,0.751912,1.718763
4,a,one,-1.291357,-0.697116


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a   -0.182965
b    0.083107
Name: data1, dtype: float64

In [7]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.183573
      two    -0.181749
b     one    -0.727009
      two     0.893223
Name: data1, dtype: float64

In [8]:
means.unstack()

key2,one,two
key1,,
a,-0.183573,-0.181749
b,-0.727009,0.893223


In [9]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -0.181749
            2006   -0.727009
Ohio        2005   -0.072015
            2006    0.670106
Name: data1, dtype: float64

In [3]:
df.groupby('key1').mean()
# 결과에 key2 column이 빠져있는 이유 : df['key2']는 숫자 데이터가 아니기 때문. 이런 column을 성가신 column이라고 부르며 결과에서 제외시킴

,data1,data2
key1,,
a,0.245717,1.437887
b,-0.189110,-0.090242


In [4]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.262147  1.563106
     two   0.212857  1.187448
b    one  -0.075660 -0.555228
     two  -0.302560  0.374745

In [5]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [6]:
# 그룹 간 순회하기
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.920586  3.317551
1    a  two  0.212857  1.187448
4    a  one -0.396291 -0.191339
b
  key1 key2    data1     data2
2    b  one -0.07566 -0.555228
3    b  two -0.30256  0.374745


In [7]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.920586  3.317551
4    a  one -0.396291 -0.191339
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.212857  1.187448
('b', 'one')
  key1 key2    data1     data2
2    b  one -0.07566 -0.555228
('b', 'two')
  key1 key2    data1     data2
3    b  two -0.30256  0.374745


In [9]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.07566,-0.555228
3,b,two,-0.30256,0.374745


In [15]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [17]:
grouped = df.groupby(df.dtypes, axis=1)

In [18]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.920586  3.317551
1  0.212857  1.187448
2 -0.075660 -0.555228
3 -0.302560  0.374745
4 -0.396291 -0.191339
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [25]:
# column이나 column의 일부만 선택하기
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one   1.563106
     two   1.187448
b    one  -0.555228
     two   0.374745

In [26]:
s_grouped = df.groupby(['key1', 'key2'])['data2']  
# 위 코드에서는 리스트나 배열인 [['data2']]라서 DataFrameGroupby 객체가 되지만 
#이 코드에서는 단일 값으로 하나의 column값인 ['data2']을 넘겼으므로 SeriesGroupBy 객체가 된다.
s_grouped.mean()

key1  key2
a     one     1.563106
      two     1.187448
b     one    -0.555228
      two     0.374745
Name: data2, dtype: float64

In [3]:
# 사전과 Series에서 그룹핑하기

people = pd.DataFrame(np.random.randn(5,5),
                     columns = ['a', 'b', 'c', 'd', 'e'],
                     index = ['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1,2]] = np.nan # nan 값 추가

people

,a,b,c,d,e
Joe,-0.146411,0.170405,0.238052,-1.420716,0.399184
Steve,-2.196687,0.072118,1.684803,0.069230,-0.286871
Wes,-0.190513,NaN,NaN,-0.046265,0.541827
Jim,0.224045,0.703461,-1.724554,0.659441,0.245896
Travis,0.520377,1.723201,-0.363872,0.261868,-0.633369


In [5]:
mapping = {'a' : 'red', 'b': 'red', 'c' : 'blue', 'd' : 'blue', 'e' : 'red', 'f' : 'orange'}

In [6]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-1.182664,0.423178
Steve,1.754033,-2.411440
Wes,-0.046265,0.351314
Jim,-1.065113,1.173402
Travis,-0.102004,1.610209


In [7]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [8]:
people.groupby(map_series, axis =1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [9]:
# 함수로 그룹핑하기

people.groupby(len).sum()

,a,b,c,d,e
3,-0.112879,0.873867,-1.486502,-0.807540,1.186907
5,-2.196687,0.072118,1.684803,0.069230,-0.286871
6,0.520377,1.723201,-0.363872,0.261868,-0.633369


In [11]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.190513  0.170405  0.238052 -1.420716  0.399184
  two  0.224045  0.703461 -1.724554  0.659441  0.245896
5 one -2.196687  0.072118  1.684803  0.069230 -0.286871
6 two  0.520377  1.723201 -0.363872  0.261868 -0.633369

In [13]:
# 색인 단계로 그룹핑하기

columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names = ['city', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4,5), columns = columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0     -0.855486  0.997568  0.640870 -0.267002 -0.658747
1     -1.056037  0.277897  0.057017  1.082439  0.110345
2     -0.338684  0.584374 -1.001209 -0.539252 -1.414086
3     -0.392615 -0.458613  1.250891 -0.964128 -0.654146

In [14]:
hier_df.groupby(level='city', axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [15]:
# 데이터 집계
# : 배열로부터 스칼라값을 만들어내는 모든 데이터 변환 작업

df

,key1,key2,data1,data2
0,a,one,0.939368,-0.504136
1,a,two,0.311258,-0.632056
2,b,one,-0.283508,-0.374722
3,b,two,0.317958,1.196366
4,a,one,0.726709,0.932695


In [16]:
grouped = df.groupby('key1')
grouped

In [17]:
grouped['data1'].quantile(0.9)

key1
a    0.896836
b    0.257811
Name: data1, dtype: float64

In [18]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)
# 자신만의 데이터 집계 함수를 사용하려면 배열의 aggregate나 agg 메서드에 해당 함수를 넣으면 된다.

,data1,data2
key1,,
a,0.628110,1.564751
b,0.601466,1.571088


In [19]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.659112  0.319464  0.311258  0.518984  0.726709  0.833039   
b      2.0  0.017225  0.425301 -0.283508 -0.133142  0.017225  0.167591   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.939368   3.0 -0.067832  0.868839 -0.632056 -0.568096 -0.504136   
b     0.317958   2.0  0.410822  1.110927 -0.374722  0.018050  0.410822   

                          
           75%       max  
key1                      
a     0.214280  0.932695  
b     0.803594  1.196366

In [28]:
# column에 여러 가지 함수 적용하기

tips = pd.read_csv('examples/pydata-book-2nd-edition/examples/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [27]:
grouped = tips.groupby(['day', 'smoker'])

In [28]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [30]:
grouped_pct.agg(['mean', 'std', peak_to_peak])
# 함수 목록이나 함수 이름을 넘기면 함수 이름을 column 이름으로 하는 DataFrame을 얻게 된다.

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [31]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])
# (name, function) 튜플의 리스트를 넘기면 각 튜플에서 첫 번째 원소가 DataFrame에서 column의 이름으로 사용된다.

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [32]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

<ipython-input-32-94f2b21cf8d2>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(functions)


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [33]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [34]:
# Input[31] 처럼 column 이름과 메서드가 담긴 튜플의 리스트를 넘기는 것도 가능하다.

ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

<ipython-input-34-7bc570689e63>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct', 'total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [35]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [36]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
            'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [37]:
# 색인되지 않은 형태로 집게된 데이터 반환하기

tips.groupby(['day', 'smoker'], as_index = False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [39]:
# Apply : 일반적인 분리-적용-병합
# apply 메서드는 객체를 여러 조각으로 나누고 전달된 함수를 각 조각에 일괄 적용한 후 합친다.

def top(df, n=5, column='tip_pct'): # tip_pct column에서 가장 큰 값을 가지는 row를 선택하는 함수
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [40]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [41]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [44]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [48]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [51]:
f = lambda x : x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

In [52]:
# 그룹 색인 생략하기
# : groupby 메서드에 group_keys = False 를 넘기면 가능하다.
tips.groupby('smoker', group_keys = False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [54]:
# 변위치 분석과 버킷 분석

frame = pd.DataFrame({'data1' : np.random.randn(1000),
                     'data2' : np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles

0       (-0.428, 1.151]
1       (-0.428, 1.151]
2      (-2.006, -0.428]
3       (-0.428, 1.151]
4       (-0.428, 1.151]
             ...       
995     (-0.428, 1.151]
996     (-0.428, 1.151]
997     (-0.428, 1.151]
998    (-2.006, -0.428]
999     (-0.428, 1.151]
Name: data1, Length: 1000, dtype: category
Categories (4, interval[float64]): [(-3.591, -2.006] < (-2.006, -0.428] < (-0.428, 1.151] < (1.151, 2.73]]

In [56]:
def get_stats(group):
    return {'min' : group.min(), 'max' : group.max(),
           'count' : group.count(), 'mean' : group.mean()}
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.591, -2.006]",-1.963857,1.062162,23.0,-0.157931
"(-2.006, -0.428]",-2.544540,3.972704,331.0,0.131603
"(-0.428, 1.151]",-2.718582,2.952673,514.0,0.017524
"(1.151, 2.73]",-2.488394,2.054243,132.0,-0.140201


In [61]:
# 변위치 숫자를 반환

grouping = pd.cut(frame.data1, 10, labels = False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-0.466561,-0.462005,2.0,-0.464283
1,-1.963857,0.948907,9.0,-0.395644
2,-2.420895,1.565284,38.0,0.034007
3,-2.544540,3.872120,129.0,0.136974
4,-1.858762,3.972704,176.0,0.144634
5,-2.718582,2.367657,226.0,-0.134715
6,-2.463493,2.952673,215.0,0.098934
7,-2.527677,2.537815,128.0,0.174731
8,-2.452375,2.054243,63.0,-0.324557


In [63]:
# 그룹에 따른 값으로 결측치 채우기

s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    0.944324
2         NaN
3    0.453970
4         NaN
5    0.370294
dtype: float64

In [64]:
s.fillna(s.mean())

0    0.589530
1    0.944324
2    0.589530
3    0.453970
4    0.589530
5    0.370294
dtype: float64

In [35]:
states = ['Ohio', 'New York', 'Vermot', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] *4 + ['West'] *4
data = pd.Series(np.random.randn(8), index = states)
data

Ohio          0.517459
New York      0.589427
Vermot        1.158946
Florida       0.970702
Oregon       -0.886148
Nevada       -1.903650
California    1.377207
Idaho         0.297256
dtype: float64

In [66]:
data[['Vermot', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.753453
New York      1.221708
Vermot             NaN
Florida      -0.058118
Oregon       -1.609710
Nevada             NaN
California   -0.136930
Idaho              NaN
dtype: float64

In [67]:
data.groupby(group_key).mean()

East    0.639014
West   -0.873320
dtype: float64

In [68]:
fill_mean = lambda g : g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.753453
New York      1.221708
Vermot        0.639014
Florida      -0.058118
Oregon       -1.609710
Nevada       -0.873320
California   -0.136930
Idaho        -0.873320
dtype: float64

In [69]:
fill_values = {'East' : 0.5, 'West' : -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          0.753453
New York      1.221708
Vermot        0.500000
Florida      -0.058118
Oregon       -1.609710
Nevada       -1.000000
California   -0.136930
Idaho        -1.000000
dtype: float64

In [2]:
# 랜덤 표본과 순열

# 하트, 스페이드, 클럽, 다이아몬드

suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1,11))+ [10] * 3) * 4
base_names = ['A'] + list(range(2,11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index = cards)

In [4]:
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

In [3]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

JC    10
3S     3
9S     9
5S     5
2D     2
dtype: int64

In [9]:
get_suit = lambda card : card[-1] # 마지막 글자가 세트
deck.groupby(get_suit).apply(draw, n=3)

C  4C      4
   8C      8
   KC     10
D  9D      9
   KD     10
   10D    10
H  QH     10
   8H      8
   5H      5
S  AS      1
   KS     10
   JS     10
dtype: int64

In [10]:
deck.groupby(get_suit, group_keys = False).apply(draw, n=2)

KC    10
QC    10
3D     3
4D     4
2H     2
AH     1
6S     6
KS    10
dtype: int64

In [15]:
# 그룹 가중 평균과 상관관계

df = pd.DataFrame({'category' : ['a', 'a', 'a',' a',
                                'b', 'b', 'b', 'b'],
                  'data' : np.random.randn(8),
                  'weights' : np.random.rand(8)})
df

,category,data,weights
0,a,-0.989119,0.947881
1,a,2.256080,0.689520
2,a,0.680068,0.420301
3,a,-0.822045,0.265442
4,b,-2.231575,0.476464
5,b,0.531641,0.647529
6,b,-1.943396,0.301190
7,b,0.672384,0.126271


In [18]:
# category별 그룹 가중 평균을 살펴보자

grouped = df.groupby('category')
get_wavg = lambda g : np.average(g['data'], weights = g['weights'])
grouped.apply(get_wavg)

category
 a   -0.822045
a     0.439266
b    -0.785999
dtype: float64

In [19]:
close_px = pd.read_csv('examples/pydata-book-2nd-edition/examples/stock_px_2.csv', parse_dates = True,
                      index_col = 0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [20]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [21]:
spx_corr = lambda x: x.corrwith(x['SPX'])
rets = close_px.pct_change().dropna()

In [22]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)

In [23]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [24]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

In [25]:
# 그룹 상의 선형회귀

import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    result = sm.OLS(Y, X).fit()
    return result.params



In [26]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX
2003,1.200841
2004,1.394995
2005,1.777145
2006,1.646548
2007,1.205186
2008,0.970660
2009,0.888944
2010,1.057903
2011,0.806352


In [29]:
# 피벗 테이블과 교차일람표

tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [30]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [31]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [32]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], 
                columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [33]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                columns = 'day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706